# Spotify Recommender Part 1 - Data Preprocessing

To do:
- check API reference for additional data that can be pulled in for artist/genres
- join data via sql
- clean data

In [123]:
import os
import re
import pandas as pd
import numpy as np
import sqlite3

from secrets import SPOTIFY_HISTORY_DB

First we setup the connection to the SQL DB which contains 3 tables:
- **streaming_history**: each song listened to starting 02-01-2021
- **genres**: each of the artists from streaming_history, tagged with their corresponding genre
- **song_features**: each track from streaming_history, with details at the song level 
    - details include corresponding track values (liveness, tempo, valence etc.)

In [3]:
conn = sqlite3.connect(SPOTIFY_HISTORY_DB)

df_sh = pd.read_sql_query("SELECT * FROM streaming_history", conn)
df_genres = pd.read_sql_query("SELECT * FROM genres", conn)
df_features = pd.read_sql_query("SELECT * FROM song_features", conn)

In [4]:
df_sh.tail(1)

,spotify_id,uri,song_name,artist_name,played_at,duration_ms,date,ETL_DTTM,artist_uri
4366,32x8lPZ4nidLsF0d0qM5KG,spotify:track:32x8lPZ4nidLsF0d0qM5KG,Heavy Heavy (feat. Tempo),Wisin,2021-09-27 00:58:43.906000,240626,2021-09-27,"2021-09-28, 00:05:09",3E6xrwgnVfYCrCs0ePERDz


In [5]:
df_genres.head(1)

,spotify_url,total_followers,genres,artist_id,artist_name,popularity,uri,ETL_DTTM
0,https://open.spotify.com/artist/3SFVIUlipGj3Rp...,48610,"['bass trap', 'traprun']",3SFVIUlipGj3RpWCKe9s73,Lox Chatterbox,58,spotify:artist:3SFVIUlipGj3RpWCKe9s73,"2021-02-14, 09:25:59"


In [6]:
df_features.head(1)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,ETL_DTTM
0,0.754,0.763,1,-6.57,1,0.295,0.32,0.0,0.143,0.704,159.963,audio_features,5ZcmLAogrj4HF8Odrz9XA2,spotify:track:5ZcmLAogrj4HF8Odrz9XA2,https://api.spotify.com/v1/tracks/5ZcmLAogrj4H...,https://api.spotify.com/v1/audio-analysis/5Zcm...,265693,3,"2021-02-04, 23:13:23"


The final query merges the data from the 3 tables into one master table, removing any potential duplicate records. If the same song was played over multiple days, there would be many.

In [196]:
merged_query = """
SELECT DISTINCT sh.*
    ,g.genres
    ,sf.*
FROM streaming_history sh 
LEFT JOIN (SELECT DISTINCT *
    FROM (SELECT DISTINCT artist_id 
                ,artist_name
                ,genres
                ,row_number() over (partition by artist_id order by artist_id, ETL_DTTM) as dup
            FROM genres)
    WHERE dup = 1) g
ON sh.artist_uri = g.artist_id
LEFT JOIN (SELECT DISTINCT id -- spotify track id
    ,danceability
    ,energy
    ,"key"
    ,loudness
    ,mode
    ,speechiness
    ,acousticness
    ,instrumentalness
    ,liveness
    ,valence
    ,tempo
    ,duration_ms
    ,time_signature
FROM song_features) sf
ON sh.spotify_id = sf.id
order by played_at
"""

df_spotify = pd.read_sql_query(merged_query, conn)

In [182]:
df_spotify.tail()

,spotify_id,uri,song_name,artist_name,played_at,duration_ms,date,ETL_DTTM,artist_uri,artist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
4362,3VkoSghZ6EeLizE8meDCX8,spotify:track:3VkoSghZ6EeLizE8meDCX8,Too Much,Dimitri Vegas & Like Mike,2021-09-27 20:13:16.067000,125714,2021-09-27,"2021-09-28, 00:05:09",73jBynjsVtofjRpdpRAJGk,73jBynjsVtofjRpdpRAJGk,...,-7.301,1.0,0.0408,0.0461,0.000174,0.163,0.332,126.014,125714.0,4.0
4363,0eV8NV4lgEbo5FGB8SxtEo,spotify:track:0eV8NV4lgEbo5FGB8SxtEo,Monster,Meldom,2021-09-27 20:27:52.996000,140184,2021-09-27,"2021-09-28, 00:05:09",0IAGPR7yj5DkqlSz7TIcsZ,0IAGPR7yj5DkqlSz7TIcsZ,...,-5.174,0.0,0.2700,0.2730,0.000002,0.397,0.389,125.145,140185.0,4.0
4364,78BOOaAvxOywAnbINpmVqe,spotify:track:78BOOaAvxOywAnbINpmVqe,Disturbia,2Hounds,2021-09-27 20:31:01.112000,187740,2021-09-27,"2021-09-28, 00:05:09",0qeQAOjEYyeV2opWZdRcU9,0qeQAOjEYyeV2opWZdRcU9,...,-5.919,0.0,0.1430,0.0071,0.000000,0.420,0.161,125.094,187740.0,4.0
4365,5VZWtVHD9X4jSpNR7PQybp,spotify:track:5VZWtVHD9X4jSpNR7PQybp,Be With You,BASTL,2021-09-27 20:33:16.775000,134565,2021-09-27,"2021-09-28, 00:05:09",2QYmb5VU9IPtOC7gs9esJB,2QYmb5VU9IPtOC7gs9esJB,...,-7.169,0.0,0.2130,0.1620,0.000015,0.109,0.113,127.923,134565.0,4.0
4366,4iN8WA2w5CYEF5ZCWoym0c,spotify:track:4iN8WA2w5CYEF5ZCWoym0c,I Don't Wanna Know,Crunkz,2021-09-27 20:36:09.575000,172323,2021-09-27,"2021-09-28, 00:05:09",4OXIfQQfNxUxqVPwoklWcv,4OXIfQQfNxUxqVPwoklWcv,...,-4.921,0.0,0.1020,0.0685,0.000000,0.401,0.387,124.125,172324.0,4.0


In [197]:
df_spotify.dtypes

spotify_id           object
uri                  object
song_name            object
artist_name          object
played_at            object
duration_ms           int64
date                 object
ETL_DTTM             object
artist_uri           object
genres               object
id                   object
danceability        float64
energy              float64
key                 float64
loudness            float64
mode                float64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms         float64
time_signature      float64
dtype: object

In [184]:
df_spotify['genres'].values[4362]

"['belgian dance', 'belgian edm', 'big room', 'dance pop', 'edm', 'electro house', 'pop dance', 'progressive electro house', 'tropical house']"

We see that the genres column appears to be a list at first glance, but is not actually in a list format:

In [185]:
df_spotify['genres'].values[4362][0]

'['

To fix this, a Regular Expression (regex) statement will be used to return a list for each genre.

In [198]:
df_spotify['genres'] = df_spotify['genres'].fillna('[]')

In [199]:
df_spotify['genres'] = df_spotify['genres'].apply(lambda x: [re.sub(' ', '_', i) for i in re.findall(r"'([^']*)'", x)])

In [208]:
df_spotify['genres'].values[4362][0]

'belgian_dance'